In [59]:
import pandas as pd

data_1 = pd.read_csv('/Users/betterhalf/Desktop/betterhalf_/pricing_prediction/Copy of Copy of Vendor sheet Image Tagging Farhan - kp.csv')
# data_2 = pd.read_csv('/Users/betterhalf/Desktop/betterhalf_/pricing_prediction/Copy of Vendor sheet Image Tagging - Prashanth (1).csv')

data_1.shape

(3859, 23)

In [60]:
data_1_final = data_1[['Image Link', 'SKU Code', 'Category', 'SKU ITEM', 'Size', 'QTY']]
# data_2_final = data_2[['Image Link', 'Category', 'SKU ITEM', 'Size', 'QTY']]

data_1_final.head(2)

,Image Link,SKU Code,Category,SKU ITEM,Size,QTY
0,https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg,16x16,BACKDROP Size,NaN,NaN,NaN
1,NaN,PF0009,Platform,Platform 1 feet height,12x12,1


In [61]:
# print(data_1_final.shape, data_2_final.shape)
# data_final = pd.concat([data_1_final, data_2_final], ignore_index=True, axis=0)
# print(data_final.shape)

In [62]:
print(data_1_final.shape)
from copy import deepcopy
data_final = deepcopy(data_1_final)

(3859, 6)


In [63]:
data_final_link = list(set(data_final['Image Link'].values))
# image_link_2 = list(set(data_2['Image Link'].values))
data_final_link[1], len(data_final_link)

('https://drive.google.com/file/d/1pps0DilwDLE7YmOW111rj5o9RN2jcq8v/view', 133)

In [64]:
final_image_link = []
for i in data_final_link:
    try:
        if i.startswith('http'):
            final_image_link.append(i)
    except:
        print(i)

# for i in image_link_2:
#     try:
#         if i.startswith('http'):
#             final_image_link.append(i)
#     except:
#         print(i)

print(len(final_image_link))

final_image_link = list(set(final_image_link))
print(len(final_image_link))


nan
132
132


In [65]:
final = []
lst = ['.jpeg', 'webp', '.jpg', '.png', '.JPG', '.JPEG', '.PNG']
for i in final_image_link:
    for j in lst:
        if j in i:
            final.append(i)
len(final)

70

In [66]:
left = []
for i in final_image_link:
    if i not in final:
        left.append(i)

left[0]

'https://drive.google.com/file/d/1pps0DilwDLE7YmOW111rj5o9RN2jcq8v/view'

In [67]:
import json

with open('left_image_links_kp.json', 'w') as f:
    json.dump(left, f)

In [68]:
import json

with open('left_image_links_kp.json', 'r') as f:
    URLS = json.load(f)

len(URLS)

62

In [ ]:
import os
import re
import csv
import json
import uuid
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, urljoin
from tqdm import tqdm

# ===================== CONFIG =====================
OUTPUT_DIR = "downloaded_images_kp"
MAPPING_CSV = "image_mapping_kp.csv"
MAPPING_JSON = "image_mapping_kp.json"
TIMEOUT = 15

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}
# ==================================================

os.makedirs(OUTPUT_DIR, exist_ok=True)
mapping = []


# ===================== HELPERS =====================
def save_mappings():
    with open(MAPPING_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["filename", "source_page_url", "image_url"]
        )
        writer.writeheader()
        writer.writerows(mapping)

    with open(MAPPING_JSON, "w", encoding="utf-8") as f:
        json.dump(mapping, f, indent=2, ensure_ascii=False)


def drive_direct_url(url):
    if "/file/d/" in url:
        return f"https://drive.google.com/uc?export=download&id={url.split('/file/d/')[1].split('/')[0]}"
    if "id=" in url:
        file_id = parse_qs(urlparse(url).query).get("id", [None])[0]
        if file_id:
            return f"https://drive.google.com/uc?export=download&id={file_id}"
    return None


def extract_pinterest_image(url):
    html = requests.get(url, headers=HEADERS, timeout=TIMEOUT).text
    soup = BeautifulSoup(html, "lxml")
    meta = soup.find("meta", property="og:image")
    return meta["content"] if meta else None


def extract_images_from_page(page_url):
    html = requests.get(page_url, headers=HEADERS, timeout=TIMEOUT).text
    soup = BeautifulSoup(html, "lxml")

    images = set()
    for img in soup.find_all("img"):
        src = img.get("src")
        if not src:
            continue

        full_url = urljoin(page_url, src)

        # 🚫 Skip Next.js static assets (logos/icons)
        if "/_next/static/media/" in full_url:
            continue

        if re.search(r"\.(jpg|jpeg|png|webp)$", full_url.lower()):
            images.add(full_url)

    return list(images)


def download_image(image_url):
    try:
        r = requests.get(image_url, headers=HEADERS, stream=True, timeout=TIMEOUT)
        r.raise_for_status()

        content_type = r.headers.get("Content-Type", "")
        ext_map = {
            "image/jpeg": ".jpg",
            "image/png": ".png",
            "image/webp": ".webp"
        }
        ext = ext_map.get(content_type.split(";")[0], ".jpg")

        filename = f"{uuid.uuid4()}{ext}"
        path = os.path.join(OUTPUT_DIR, filename)

        with open(path, "wb") as f:
            for chunk in r.iter_content(1024):
                f.write(chunk)

        return filename

    except Exception as e:
        print(f"❌ Failed: {image_url} → {e}")
        return None


def save_mapping_entry(filename, page_url, image_url):
    mapping.append({
        "filename": filename,
        "source_page_url": page_url,
        "image_url": image_url
    })


# ===================== MAIN LOGIC =====================
def process_url(page_url):
    print(f"🔗 Processing: {page_url}")

    # Direct image
    if re.search(r"\.(jpg|jpeg|png|webp)$", page_url.lower()):
        saved = download_image(page_url)
        if saved:
            save_mapping_entry(saved, page_url, page_url)
        return

    # Google Drive
    if "drive.google.com" in page_url:
        direct = drive_direct_url(page_url)
        if direct:
            saved = download_image(direct)
            if saved:
                save_mapping_entry(saved, page_url, direct)
        return

    # Pinterest
    if "pinterest." in page_url:
        img_url = extract_pinterest_image(page_url)
        if img_url:
            saved = download_image(img_url)
            if saved:
                save_mapping_entry(saved, page_url, img_url)
        return

    # Web page (TheWeddingCompany, etc.)
    image_urls = extract_images_from_page(page_url)
    for image_url in image_urls:
        saved = download_image(image_url)
        if saved:
            save_mapping_entry(saved, page_url, image_url)



for url in tqdm(URLS, desc="Downloading images"):
    process_url(url)

save_mappings()

# Safety check
assert not any(m["filename"].startswith("page_") for m in mapping)

print("\n✅ DONE")
print(f"📁 Images saved to: {OUTPUT_DIR}")
print(f"🗂 Mapping files: {MAPPING_CSV}, {MAPPING_JSON}")


In [ ]:
import json

with open('image_mapping_kp.json', 'r') as f:
    data = json.load(f)

In [69]:
data[0]

{'filename': 'c8bbcdd7-e964-4406-aac2-62ab782c4898.jpg',
 'source_page_url': 'https://www.theweddingcompany.com/wedding-ideas/idea/romantic-floral-haven-72C917D6-EE49-47A0-B415-41598FFD95DD',
 'image_url': 'https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/72C917D6-EE49-47A0-B415-41598FFD95DD.jpeg'}

In [70]:
data_final.head(2)

,Image Link,SKU Code,Category,SKU ITEM,Size,QTY
0,https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg,16x16,BACKDROP Size,NaN,NaN,NaN
1,NaN,PF0009,Platform,Platform 1 feet height,12x12,1


In [82]:
data_final['Image Link'].fillna('EMPTY', inplace=True)
data_final['Category'].fillna('EMPTY', inplace=True)
data_final['SKU ITEM'].fillna('EMPTY', inplace=True)
data_final['Size'].fillna('EMPTY', inplace=True)
data_final['QTY'].fillna('EMPTY', inplace=True)


/var/folders/m3/37phlr4d0yg2r9r8f8x3kn6h0000gq/T/ipykernel_38005/3899169860.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_final['Size'].fillna('EMPTY', inplace=True)
/var/folders/m3/37phlr4d0yg2r9r8f8x3kn6h0000gq/T/ipykernel_38005/3899169860.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [72]:
type(lst[-2]), lst[-2]

(str, '.JPEG')

In [73]:
Link = data_final['Image Link'].tolist()
cat = data_final['Category'].tolist()
SKU_ITEM = data_final['SKU ITEM'].tolist()
lst = ['.jpeg', 'webp', '.jpg', '.png', '.JPG', '.JPEG', '.PNG']

final_links = []
for i in range(len(Link)):
    if Link[i] != 'EMPTY' and Link[i].endswith(tuple(lst)):
        current_link = Link[i]
        final_links.append(current_link)

    elif Link[i] != 'EMPTY':
        found = False
        for item in data:
            if item['source_page_url'] == Link[i]:
                final_links.append(item['image_url'])
                found = True
                break
        if not found:
            print(Link[i])
            final_links.append('EMPTY')

    else:
        if cat[i] != 'EMPTY' and SKU_ITEM[i] != 'EMPTY':
            final_links.append(current_link)
        else:
            final_links.append('EMPTY')

In [74]:
data_final['FINAL_LINK'] = final_links

In [78]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
data_final.head(2)

,Image Link,SKU Code,Category,SKU ITEM,Size,QTY,FINAL_LINK
0,https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg,16x16,BACKDROP Size,EMPTY,NaN,NaN,https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg
1,EMPTY,PF0009,Platform,Platform 1 feet height,12x12,1,https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg


In [77]:
data_final.to_csv('final_data_kp.csv', index=False)

In [ ]:
final_data = {}
done = []
mm = 0

for i in data_final.iterrows():
    xx = dict(i[1])
    print(xx)
    if xx['FINAL_LINK'] == 'EMPTY':
        continue
    final_data['Image Link'] = xx['FINAL_LINK']
    if xx['FINAL_LINK'] not in done:
        final_data[xx['FINAL_LINK']] = {
            'Platform': [],
            'Structures': [],
            'Furniture': [],
            'Flowers': [],
            'Props&Accessorie': [],
            'Lighting': [],
            'Fabric': [],

        }
        done.append(xx['FINAL_LINK'])
    if xx['Category'] == 'BACKDROP Size':
        final_data[xx['FINAL_LINK']]['backdrop_size'] = xx['SKU Code']
        continue
    if xx['Category'] in ['Real flowers', 'Artificial Flowers', 'Real Flowers']:
        new_category = 'Flowers'
    else:
        new_category = xx['Category']
    dic = {
        'SKU ITEM': xx['SKU ITEM'],
        'Size': xx['Size'],
        'QTY': xx['QTY']
    }
    final_data[xx['FINAL_LINK']][new_category].append(dic)

    if mm == 50:
        break
    mm += 1



{'Image Link': 'https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg', 'SKU Code': '16x16', 'Category': 'BACKDROP Size', 'SKU ITEM': 'EMPTY', 'Size': 'EMPTY', 'QTY': 'EMPTY', 'FINAL_LINK': 'https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg'}
{'Image Link': 'EMPTY', 'SKU Code': 'PF0009', 'Category': 'Platform', 'SKU ITEM': 'Platform 1 feet height', 'Size': '12x12', 'QTY': '1', 'FINAL_LINK': 'https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg'}
{'Image Link': 'EMPTY', 'SKU Code': 'PF0018', 'Category': 'Platform', 'SKU ITEM': 'carpet', 'Size': '12x12', 'QTY': '1', 'FINAL_LINK': 'https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg'}
{'Image Link': 'EMPTY', 'SKU Code': 'PF0013', 'Category': 'Pl

KeyError: 'Real flowers'

In [92]:
final_data

{'Image Link': 'https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg',
 'https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg': {'Platform': [{'SKU ITEM': 'Platform 1 feet height',
    'Size': '12x12',
    'QTY': '1'}],
  'Structures': [],
  'Furniture': [],
  'Flowers': [],
  'prop_and_accessories': [],
  'Lighting': [],
  'Fabric': [],
  'backdrop_size': '16x16'}}

In [89]:
j

'Image Link'

In [84]:
k = 0

for i in data_final.iterrows():
    xx = dict(i[1])
    print(xx)
    for j in xx:
        print(j)
    break

{'Image Link': 'https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg', 'SKU Code': '16x16', 'Category': 'BACKDROP Size', 'SKU ITEM': 'EMPTY', 'Size': 'EMPTY', 'QTY': 'EMPTY', 'FINAL_LINK': 'https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg'}
Image Link
SKU Code
Category
SKU ITEM
Size
QTY
FINAL_LINK


In [87]:
k = 0

for k, i in enumerate(data_final.iterrows()):
    xx = dict(i[1])
    print(k, xx)
    k += 1
    if k == 50:
        break

0 {'Image Link': 'https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg', 'SKU Code': '16x16', 'Category': 'BACKDROP Size', 'SKU ITEM': 'EMPTY', 'Size': 'EMPTY', 'QTY': 'EMPTY', 'FINAL_LINK': 'https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg'}
1 {'Image Link': 'EMPTY', 'SKU Code': 'PF0009', 'Category': 'Platform', 'SKU ITEM': 'Platform 1 feet height', 'Size': '12x12', 'QTY': '1', 'FINAL_LINK': 'https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg'}
2 {'Image Link': 'EMPTY', 'SKU Code': 'PF0018', 'Category': 'Platform', 'SKU ITEM': 'carpet', 'Size': '12x12', 'QTY': '1', 'FINAL_LINK': 'https://imageswedding.theweddingcompany.com/bh_prod_bucket/watermark_assets/ideabook/F43A7F8C-2A7E-4253-AEA2-573A7F93CFA3.jpeg'}
3 {'Image Link': 'EMPTY', 'SKU Code': 'PF0013', 'Catego